In [1]:
import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
dir_path = '../input/garbage classification/Garbage classification'

In [3]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [4]:
len(img_list)

2527

In [5]:
train=ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.1,zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)
test=ImageDataGenerator(rescale=1/255,validation_split=0.1)
train_generator=train.flow_from_directory(dir_path,target_size=(300,300),batch_size=32,
                                          class_mode='categorical',subset='training')
test_generator=test.flow_from_directory(dir_path,target_size=(300,300),batch_size=32,
                                        class_mode='categorical',subset='validation')
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [6]:
model=Sequential()
    
model.add(Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(6,activation='softmax'))

filepath="trained_model.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]



Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43808)             0         
__________

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [9]:
model.fit_generator(train_generator, epochs=100, steps_per_epoch=2276//32,validation_data=test_generator,
                    validation_steps=251//32,callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
71/71 [==============================] - 57s 805ms/step - loss: 1.6133 - acc: 0.3195 - val_loss: 1.3988 - val_acc: 0.4196

Epoch 00001: val_acc improved from -inf to 0.41964, saving model to trained_model.h5
Epoch 2/100
71/71 [==============================] - 52s 730ms/step - loss: 1.3469 - acc: 0.4498 - val_loss: 1.3363 - val_acc: 0.4521

Epoch 00002: val_acc improved from 0.41964 to 0.45205, saving model to trained_model.h5
Epoch 3/100
71/71 [==============================] - 52s 732ms/step - loss: 1.2552 - acc: 0.5027 - val_loss: 1.5238 - val_acc: 0.4521

Epoch 00003: val_acc did not improve from 0.45205
Epoch 4/100
71/71 [==============================] - 52s 728ms/step - loss: 1.2640 - acc: 0.5070 - val_loss: 1.2740 - val_acc: 0.5068

Epoch 00004: val_acc improved from 0.45205 to 0.50685, saving model to trained_model.h5
Epoch 5/100
71/71 [==============================] - 52s 730ms/step - loss: 1.1163 - acc: 0.5739 - va